<a href="https://colab.research.google.com/github/lddm/forests-monitoring/blob/main/GEE_LS8_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
        import geemap

try:
        import geetools
except ImportError:
        print('geetools package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geetools'])
        import geetools


geetools package not installed. Installing ...


In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [36]:
ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# Initial date of interest (inclusive).
i_date = '2015-01-01'

# Final date of interest (exclusive).
f_date = '2015-02-28'

rgb_bands = ['B4', 'B3', 'B2']

ls8 = ls8.select(rgb_bands).filterDate(i_date, f_date)

In [37]:
roi_center_lon = -55.596655
roi_center_lat = -31.388125
offset = 0.05
roi = ee.Geometry.Rectangle(roi_center_lon - offset,
                      roi_center_lat - offset,
                      roi_center_lon + offset,
                      roi_center_lat + offset)

print('Before ROI filtering:', ls8.size().getInfo())
ls8 = ls8.filterBounds(roi)
print('After ROI filtering:', ls8.size().getInfo())

Before ROI filtering: 17665
After ROI filtering: 7


In [38]:
# This is OK for small collections
# print(ls8.size())
if ls8.size().getInfo() == 0:
  raise ValueError('No rasters for the selected ROI and date range were found')
ls8_collection_list = ls8.toList(ls8.size())
collection_size = ls8_collection_list.size().getInfo()

map = geemap.Map()
ls8_vis = {'min': 0, 'max': 3000, 'opacity': 1.0, 'gamma': 1.4}
map.set_center(-54.8712, -31.7001, 8)

for i in range(collection_size):
  raster_img = ee.Image(ls8_collection_list.get(i))
  map.addLayer(raster_img, ls8_vis, 'LS8-' + str(i))
map.addLayerControl()
map

### Export to Drive

In [42]:
help(ee.batch.Export.image.toDrive)

Help on function toDrive in module ee.batch:

toDrive(image, description='myExportImageTask', folder=None, fileNamePrefix=None, dimensions=None, region=None, scale=None, crs=None, crsTransform=None, maxPixels=None, shardSize=None, fileDimensions=None, skipEmptyTiles=None, fileFormat=None, formatOptions=None, **kwargs)
    Creates a task to export an EE Image to Drive.
    
    Args:
      image: The image to be exported.
      description: Human-readable name of the task.
      folder: The name of a unique folder in your Drive account to
          export into. Defaults to the root of the drive.
      fileNamePrefix: The Google Drive filename for the export.
          Defaults to the name of the task.
      dimensions: The dimensions of the exported image. Takes either a
          single positive integer as the maximum dimension or "WIDTHxHEIGHT"
          where WIDTH and HEIGHT are each positive integers.
      region: The lon,lat coordinates for a LinearRing or Polygon
          speci

In [45]:
bands = ['B2', 'B3', 'B4']
scale = 30
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
date_pattern = 'ddMMMy' # dd: day, MMM: month (JAN), y: year
folder = 'LS8_captures'
data_type = 'uint32'
extra = dict(sat='L8SR')
region = roi

In [46]:

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=ls8,
            folder=folder,
            region=site,
            namePattern=name_pattern,
            scale=scale,
            dataType=data_type,
            datePattern=date_pattern,
            extra=extra,
            verbose=True,
            maxPixels=int(1e13)
        )

exporting L8SR_06Jan2015_223-82 to folder 'LS8_captures' in GDrive
exporting L8SR_22Jan2015_223-82 to folder 'LS8_captures' in GDrive
exporting L8SR_07Feb2015_223-82 to folder 'LS8_captures' in GDrive
exporting L8SR_23Feb2015_223-82 to folder 'LS8_captures' in GDrive
exporting L8SR_13Jan2015_224-82 to folder 'LS8_captures' in GDrive
exporting L8SR_29Jan2015_224-82 to folder 'LS8_captures' in GDrive
exporting L8SR_14Feb2015_224-82 to folder 'LS8_captures' in GDrive
